# Malaria Detection using Deep Learning


https://www.kaggle.com/iarunava/cell-images-for-detecting-malaria

### Importing necessary libraries

In [1]:
pwd

'/home/wsuser/work'

In [7]:
!pip install keras==2.2.4
!pip install tensorflow


     |████████████████████████████████| 312 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 10.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 5.4 MB/s  eta 0:00:01
     |████████████████████████████████| 3.8 MB 37.3 MB/s eta 0:00:01
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.35.0
    Uninstalling grpcio-1.35.0:
      Successfully uninstalled grpcio-1.35.0
  Attempting uninstall: opt-einsum
    Found existing installation: opt-einsum 3.1.0
    Uninstalling opt-einsum-3.1.0:
      Successfully uninstalled opt-einsum-3.1.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 20210226132247
    Uninstalling flatbuffers-20210226132247:
      Successfully uninstalled flatbuffers-20210226132247


In [8]:
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras.layers import Dense#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Flatten#used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D#Convolutional layer
from tensorflow.keras.layers import MaxPool2D# used for downsampling of the input
from tensorflow.keras.layers import Dropout #used for preventing overfitting of model
from tensorflow.keras.preprocessing.image import ImageDataGenerator# to artificially expand the size of a training dataset
import numpy as np#used for numerical analysis

In [9]:
img_width=64 #setting default image width size 
img_height=64 #setting default image height size

### Image Data Agumentation

In [10]:
datagen=ImageDataGenerator(rescale=1/255.0,validation_split=0.2) #Image data augmentation

In [11]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_8414f7cddadb4efab338581058acebb1 = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_8414f7cddadb4efab338581058acebb1 = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

client_8414f7cddadb4efab338581058acebb1 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ZC-5mqH1rnr6iRSGomD7AY73Qj_0hsxr4Pci5zVwWScC',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_8414f7cddadb4efab338581058acebb1)

streaming_body_2 = client_8414f7cddadb4efab338581058acebb1.get_object(Bucket='artificialintelligence-donotdelete-pr-e4khfcgmdiynsa', Key='archive (1) (1).zip')['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [12]:
from io import BytesIO
import zipfile
unzip=zipfile.ZipFile(BytesIO(streaming_body_2.read()),'r')
file_paths=unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [13]:
pwd

'/home/wsuser/work'

In [14]:
import os
filenames=os.listdir('/home/wsuser/work/cell_images')

In [15]:
train_data_generator=datagen.flow_from_directory(directory='/home/wsuser/work/cell_images',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="training")#applying image data agumentation to training data

Found 44095 images belonging to 3 classes.


In [16]:
validation_data_generator=datagen.flow_from_directory(directory='/home/wsuser/work/cell_images',
                                                target_size=(img_width,img_height),class_mode="binary",
                                                batch_size=16,subset="validation")#applying image data agumentation to testing data

Found 11021 images belonging to 3 classes.


In [17]:
train_data_generator.class_indices

{'Parasitized': 0, 'Uninfected': 1, 'cell_images': 2}

In [12]:
train_data_generator.labels #checking the labels

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

### Creating the Model

In [18]:
model= Sequential() #our model

model.add(Conv2D(16,(3,3),input_shape=(img_width,img_height,3),activation="relu"))#convolutional layer
model.add(MaxPool2D(2,2)) # Downsamples the input
model.add(Dropout(0.2)) #drop input randomly for preventing from overfitting

model.add(Conv2D(32,(3,3),activation="relu"))
model.add(MaxPool2D(2,2))
model.add(Dropout(0.3))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(64,activation="relu"))#deeply connected neural network layer
model.add(Dropout(0.5))

model.add(Dense(1,activation="sigmoid")) #output layer with one neuron


In [14]:
model.summary() #summary of our neural network

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 16)        448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 31, 31, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 6272)              0

### Compiling the model

In [19]:
#compile modelb
model.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])

### Fitting the model

In [20]:
history = model.fit_generator(generator=train_data_generator,
                              steps_per_epoch = len(train_data_generator),
                              epochs = 5,
                              validation_data = validation_data_generator,
                              validation_steps = len(validation_data_generator))

/opt/conda/envs/Python-3.8-main/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1839: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/5
2756/2756 [==============================] - 297s 108ms/step - loss: -1065035619.0898 - accuracy: 0.2502 - val_loss: -20435890176.0000 - val_accuracy: 0.2500- ETA: 23s - loss: -766945995.8182 - accu - ETA: 23s - loss: -773577319.7552 - accuracy: 0. - ETA: 22s - loss: -775795417.0676 - accuracy: 0. - ETA: 22s - loss: -778 - ETA: 21s - loss: -7 -  - ETA: 8s - loss: -952049817.4995 - accuracy: 0 - ETA: 7s - loss: -958437895.4735 - accuracy: 0.250 - ETA: 7s - loss: -959716771.862
Epoch 2/5
2756/2756 [==============================] - 296s 107ms/step - loss: -49331364433.8977 - accuracy: 0.2517 - val_loss: -193704296448.0000 - val_accuracy: 0.2500accuracy: 0.239 - ETA: 4:25 - loss: -23658414690.4615 - accu - ETA: 4:24 - loss: -24026896208.0000 - accuracy: 0 - ETA: 4:24 - loss: -24166366920.3478 - accuracy: 0. - ETA: 4:24 - loss: -24258317 - ETA: 4:22 - loss: -24439486617.6000 - accuracy: 0. - ETA: 4:22 - loss: -24433104177.2308 -  - ETA: 4:13 - loss: -24562453677.5593 - accuracy: 

### Saving our model

In [21]:
from tensorflow.keras.models import load_model

model.save('maleria.h5')

In [22]:
!tar -zcvf image-classification-model_new.tgz maleria.h5

maleria.h5


In [23]:
ls -1

cell_images/
image-classification-model_new.tgz
maleria.h5


In [24]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 22.8 MB/s eta 0:00:01


In [25]:
from ibm_watson_machine_learning import APIClient
wml_credentials={
    "url":"https://us-south.ml.cloud.ibm.com",
    "apikey":"gvmh7n76xCp4Zwdyvhh8Ab6WwSftku7r_BCRN3BMA-Dm"
}
client = APIClient(wml_credentials)


In [26]:
client = APIClient(wml_credentials)


In [29]:
def guid_from_space_name(client,space_name):
    space=client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [30]:
space_uid=guid_from_space_name(client,'maleria_deploy')
print("Space UID="+space_uid)

Space UID=e31b2240-6adb-4213-aa34-0359ba3ed59e


In [31]:
client.set.default_space(space_uid)

'SUCCESS'

In [26]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
scikit-learn_0.22-py3.6        154010fa-5b3b-4ac1-82af-4d5ee5abbc85  base
default_r3.6                   1b70aec3-ab34-4b87-8aa0-a4a3c8296a36  base
pytorch-onnx_1.3-py3.6         1bc6029

In [32]:
#software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_1.15-py3.6")
software_spec_uid=client.software_specifications.get_uid_by_name("tensorflow_2.4-py3.7")
software_spec_uid

'65e171d7-72d1-55d9-8ebb-f813d620c9bb'

In [28]:
# import tensorflow
# tensorflow.__version__

In [29]:
# !pip install keras==2.2.4

In [33]:
model_details = client.repository.store_model(model='image-classification-model_new.tgz',meta_props={
    client.repository.ModelMetaNames.NAME:"CNN",
    client.repository.ModelMetaNames.TYPE:"keras_2.2.4",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_spec_uid
})
model_id = client.repository.get_model_uid(model_details)

Note: Warnings!! :  Model type keras_2.2.4 is deprecated. We recommend you use a supported model type. See Supported Frameworks https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/pm_service_supported_frameworks.html


In [31]:
model_id

'e10d376d-ed76-4642-9dc6-0321c3f2e733'

### Predicting our results

In [32]:
client.repository.download(model_id,'my_model7.tar.gz')

Successfully saved model content to file: 'my_model7.tar.gz'


'/home/wsuser/work/my_model7.tar.gz'

In [33]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
model = load_model('maleria.h5') #loading our trained model

In [34]:
y_pred = model.predict(validation_data_generator) #performing prediction to the test data

In [35]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)#Returns the indices of the maximum values along an axis
y_pred

array([0, 0, 0, ..., 0, 0, 0])